In [ ]:
from selenium import webdriver
import requests
import time

# 模拟浏览器遍历车型信息
# 设置请求头
options = webdriver.ChromeOptions()
options.add_argument('User-Agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.6 Safari/537.36"')
options.add_argument('Accept="text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3"')
options.add_argument('Origin="https://www.tuhu.cn"')

ip = '122.114.125.90'
port = '16819'
options.add_argument('--proxy-server=http://%s:%s' % (ip, port))

driver = webdriver.Chrome(chrome_options=options)
# 浏览器窗口最大化
driver.maximize_window()

url = 'https://www.tuhu.cn/'
driver.get(url)
time.sleep(5)

# 车型选择弹框
driver.find_element_by_xpath('.//div[@id="myCar"]').click()
time.sleep(3)

# 车型选择弹框 - 所有内容
CarSelect = driver.find_element_by_xpath('.//div[@id="carSelect"]')

# 车型选择弹框 - 品牌首字母
CarZiMus = CarSelect.find_element_by_xpath('.//div[@id="div2"]').find_element_by_tag_name('ul').find_elements_by_tag_name('li')







In [ ]:


# 字母层级
for letter in CarZiMus:
    if len(letter.text) == 1:
        letter.click() # 点击字母
        time.sleep(2)
        
        # 品牌层级
        CarBrands = driver.find_element_by_xpath('.//div[@id="CarBrands"]').find_elements_by_tag_name('li')
        for brand in CarBrands:
            
            # 提取信息：品牌
            OriginalBrand = brand.get_attribute('data-brand')
            Brand = brand.get_attribute('data-brand')[4:]
            # print(OriginalBrand,Brand)
            
            brand.click() # 点击品牌
            time.sleep(2)
            
            # 车系层级
            Series = driver.find_element_by_xpath('.//div[@id="div5"]').find_elements_by_xpath('.//li[not(contains(@class,"CarBrandTitle"))]')
            Series_len = len(Series)
            for series_index in range(len(Series)):
                series = driver.find_element_by_xpath('.//div[@id="div5"]').find_elements_by_xpath('.//li[not(contains(@class,"CarBrandTitle"))]')
                
                # 提取信息：车系ID，车系-品牌厂商，车系
                SeriesID = series[series_index].get_attribute('data-id')
                SeriesAndFactory = series[series_index].get_attribute('data-vehicle')
                SeriesName = series[series_index].text
                Factory = SeriesAndFactory.replace(SeriesName+'-','')
                # print(SeriesID,SeriesAndFactory,SeriesName)
                
                series[series_index].click() # 点击车系
                time.sleep(2)
                
                try:
                    # 排量层级
                    Displacements = driver.find_element_by_xpath('.//div[@id="div5"]').find_elements_by_tag_name('li')
                    Displacements_len = len(Displacements)

                    for displacement_index in range(Displacements_len):
                        displacements = driver.find_element_by_xpath('.//div[@id="div5"]').find_elements_by_tag_name('li')

                        # 提取排量名称
                        DisplacementName = displacements[displacement_index].get_attribute('data-pailiang')
                        DisplacementText = displacements[displacement_index].text
                        if DisplacementName == DisplacementText:
                            pass
                        else:
                            print('品牌：%s  车系：%s  排量：%s  排量属性取值与文本取值不一致' % (OriginalBrand,SeriesName,DisplacementName))

                        displacements[displacement_index].click() # 点击排量
                        time.sleep(3)
                        
                        try:
                            # 年份层级
                            YearProduct = driver.find_element_by_xpath('.//div[@id="div5"]').find_elements_by_tag_name('li')
                            for years in YearProduct:
                                Years = years.get_attribute('data-nian')
                                # print(Years)
                                maintenance_url = 'https://by.tuhu.cn/baoyang/' + SeriesID + '/pl' + DisplacementName + '-n' + str(Years) + '.html'
                                result_list = [OriginalBrand,Brand,SeriesID,SeriesName,SeriesAndFactory,Factory,DisplacementName,Years,maintenance_url]
                                print(result_list)
                                # print(maintenance_url)
                        except:
                            print('品牌：%s  车系：%s  排量：%s  年份选项未采集完整' % (OriginalBrand,SeriesName,DisplacementName))
                            driver.find_element_by_xpath('.//div[@data-index="3"]').click()
                            time.sleep(2)
                            continue

                        driver.find_element_by_xpath('.//div[@data-index="3"]').click() # 关闭排量
                        time.sleep(2)
                except:
                    print('品牌：%s  车系：%s  排量选项未采集完整' % (OriginalBrand,SeriesName))
                    driver.find_element_by_xpath('.//div[@data-index="2"]').click()
                    time.sleep(2)
                    continue
                    
                driver.find_element_by_xpath('.//div[@data-index="2"]').click() # 关闭车系
                time.sleep(2)
                
            driver.find_element_by_xpath('.//div[@data-index="1"]').click() # 关闭品牌
            time.sleep(2)
            

    

In [13]:
from selenium import webdriver
import requests
import time
import pymysql
from selenium.common.exceptions import NoSuchElementException
# 模拟访问途虎养车首页
def www_tuhu_cn(url='https://www.tuhu.cn/'):
    # 创建会话
    options = webdriver.ChromeOptions()
    # 设置请求头
    options.add_argument('User-Agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.6 Safari/537.36"')
    options.add_argument('Accept="text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3"')
    options.add_argument('Origin="https://www.tuhu.cn"')
    # options.add_argument('--headless')
    driver = webdriver.Chrome(chrome_options=options)
    # 浏览器窗口最大化
    driver.maximize_window()
    # 模拟访问目标网址
    driver.get(url)
    # 硬等待
    time.sleep(20)
    return driver

# 手动点击车型选择弹框
def click_carSelect(driver):
    driver.find_element_by_xpath('.//div[@id="myCar"]').click()
    time.sleep(3)
    return driver

driver = www_tuhu_cn(url='https://www.tuhu.cn/')
driver = click_carSelect(driver)

# 车型选择弹框 - 所有内容
CarSelect = driver.find_element_by_xpath('.//div[@id="carSelect"]')
# 车型选择弹框 - 品牌首字母
CarZiMus = CarSelect.find_element_by_xpath('.//div[@id="div2"]').find_element_by_tag_name('ul').find_elements_by_tag_name('li')


for letter in CarZiMus:
    if len(letter.text) == 1 and letter.text == 'S':
        letter.click()  # 点击字母
        time.sleep(2)
        
        # 品牌层级
        CarBrands = driver.find_element_by_xpath('.//div[@id="CarBrands"]').find_elements_by_tag_name('li')
        for brand in CarBrands:
            # 提取信息：品牌
            OriginalBrand = brand.get_attribute('data-brand')
            Brand = brand.get_attribute('data-brand')[4:]
            # print(OriginalBrand,Brand)
            if Brand == '萨博':
                brand.click()  # 点击品牌
                time.sleep(2)
                
                # 车系层级
                Series = driver.find_element_by_xpath('.//div[@id="div5"]').find_elements_by_xpath('.//li[not(contains(@class,"CarBrandTitle"))]')
                for series_index in range(len(Series)):
                    series = driver.find_element_by_xpath('.//div[@id="div5"]').find_elements_by_xpath('.//li[not(contains(@class,"CarBrandTitle"))]')

                    # 提取信息：车系ID，车系-品牌厂商，车系
                    SeriesID = series[series_index].get_attribute('data-id')
                    SeriesAndFactory = series[series_index].get_attribute('data-vehicle')
                    SeriesName = series[series_index].text
                    Factory = SeriesAndFactory[SeriesAndFactory.find('-') + 1:]
                    # print(SeriesID,SeriesAndFactory,SeriesName)
                    
                    if SeriesName == '9000':
                        series[series_index].click()  # 点击车系
                        time.sleep(5)
                        
                        # 点击车系后，判断车型选择弹框是否存在
                        # 1.存在，继续，并判断是否需要再次选择车系
                        # 2.不存在，则该品牌车系下无更细销售型，重新点击车系选择弹框
                        try:
                            carSelect_element_exitOrNot = driver.find_element_by_xpath('.//div[@id="carSelect"]')
                        except NoSuchElementException as e:
                            print(e)
                            driver = click_carSelect(driver)
                        
                 
                            


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@id="carSelect"]"}
  (Session info: chrome=74.0.3729.6)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.18362 x86_64)



StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=74.0.3729.6)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.18362 x86_64)


In [20]:
driver = www_tuhu_cn(url='https://www.tuhu.cn/')
driver = click_carSelect(driver)
CarSelect = driver.find_element_by_xpath('.//div[@id="carSelect"]')
print(CarSelect)
print('-'*40)
time.sleep(5)
driver.refresh()
time.sleep(5)
driver = click_carSelect(driver)
CarSelect = driver.find_element_by_xpath('.//div[@id="carSelect"]')
print(CarSelect)
# driver.close()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


<selenium.webdriver.remote.webelement.WebElement (session="6e58fa6eac575a20d382185f1c7dc920", element="0.20411409791141355-2")>
----------------------------------------
<selenium.webdriver.remote.webelement.WebElement (session="6e58fa6eac575a20d382185f1c7dc920", element="0.8583395978195003-2")>
